In [ ]:
# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split

In [29]:
# Download dataset (run only once!)
# !wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

In [30]:
# Read csv file
df = pd.read_csv('course_lead_scoring.csv')

In [31]:
# Data preparation
# Check if the missing values are presented in the features. If there are missing values:
df.dtypes
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [32]:
# For categorical features, replace them with 'NA', and for numerical features, replace with with 0.0
df = df.fillna({col: 'NA' for col in df.select_dtypes('object')})
df = df.fillna(0.0)
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [33]:
# Question 1: What is the most frequent observation (mode) for the column industry?
df['industry'].mode()[0]

'retail'

In [35]:
# Question 2: Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.
# What are the two features that have the biggest correlation?
num_df = df.select_dtypes(include=['number'])
corr = num_df.corr()
corr

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879,0.435914
annual_income,0.009770,1.000000,0.027036,0.015610,0.053131
interaction_count,-0.023565,0.027036,1.000000,0.009888,0.374573
lead_score,-0.004879,0.015610,0.009888,1.000000,0.193673
converted,0.435914,0.053131,0.374573,0.193673,1.000000


In [ ]:
# Split your data in train/val/test sets with 60%/20%/20% distribution.
# Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
# Make sure that the target value y is not in your dataframe.
X = df.drop(columns=['converted'])
y = df['converted']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

len(X_train) / len(df), len(X_val) / len(df), len(X_test) / len(df)


(0.5998632010943913, 0.1997264021887825, 0.20041039671682626)

In [ ]:
# Question 3: Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only. Round the scores to 2 decimals using round(score, 2).